In [2]:
import os
import json
import pandas as pd
import pickle
import uuid
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm, trange
import urllib
from concurrent.futures import ThreadPoolExecutor, as_completed
import networkx as nx
import numpy as np
from collections import Counter
from SPARQLWrapper import SPARQLWrapper, JSON

# !label-studio start

# set the maximum number of retries
MAX_RETRIES = 10

# load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

uuid = str(uuid.uuid4()).split("-")[-1]
uuid

'667e63208a59'

Read the queries from CSV

In [3]:
df = pd.read_csv("filtered_questions_63a0f8a06513.csv", index_col=0)
df["dbpedia_entities"] = df["dbpedia_entities"].apply(lambda x: eval(x))
df["placeholders"] = df["placeholders"].apply(lambda x: eval(x))
if isinstance(df["dbpedia_entities"].iloc[0], dict):
    df["dbpedia_entities_re"] = df["dbpedia_entities"].apply(
    lambda x: {k: v.split("/")[-1].split("#")[-1] for k, v in x.items()}
)
else:
    df["dbpedia_entities_re"] = df["dbpedia_entities"].apply(
        lambda x: {k: v for k, v in x.items()}
    )
df

,question,type,placeholders,naturalness,difficulty,dbpedia_entities,dbpedia_entities_re
0,Please describe Ada Lovelace’s contributions t...,character description,{'person': 'Ada Lovelace'},high,medium,{'person': 'http://dbpedia.org/resource/Ada_Lo...,{'person': 'Ada_Lovelace'}
1,Provide a detailed description of the backgrou...,event description,{'event': 'Moon landing in 1969'},high,medium,{'event': 'http://dbpedia.org/resource/Apollo_...,{'event': 'Apollo_11'}
2,Explain the relationship between the Roman Emp...,relationship explanation,"{'entity A': 'Roman Empire', 'entity B': 'Han ...",high,hard,{'entity A': 'http://dbpedia.org/resource/Roma...,"{'entity A': 'Roman_Empire', 'entity B': 'Han_..."
3,"Based on Amazon's historical behavior, in whic...",trend prediction,{'entity': 'Amazon'},high,medium,{'entity': 'http://dbpedia.org/resource/Amazon...,{'entity': 'Amazon_(company)'}
4,Please describe Marie Curie's contributions to...,character description,{'person': 'Marie Curie'},high,medium,{'person': 'http://dbpedia.org/resource/Marie_...,{'person': 'Marie_Curie'}
...,...,...,...,...,...,...,...
845,Compare and contrast the similarities and diff...,contrast analysis,"{'entity A': 'Impressionist art', 'entity B': ...",high,medium,{'entity A': 'http://dbpedia.org/resource/Impr...,"{'entity A': 'Impressionism', 'entity B': 'Pos..."
846,Compare and contrast the similarities and diff...,contrast analysis,"{'entity A': 'New Deal', 'entity B': 'Great So...",high,hard,{'entity A': 'http://dbpedia.org/resource/New_...,"{'entity A': 'New_Deal', 'entity B': 'Great_So..."
847,Compare the impact of the American Revolutiona...,historical comparison,{'historical event A': 'American Revolutionary...,high,hard,{'historical event A': 'http://dbpedia.org/res...,{'historical event A': 'American_Revolutionary...
848,How do you evaluate the impact of the Internet...,evaluation and reflection,"{'person/event': 'Internet', 'field': 'global ...",high,medium,{'person/event': 'http://dbpedia.org/resource/...,"{'person/event': 'Internet', 'field': 'Commerce'}"


In [4]:
# # remove the questions that have error subgraphs
# with open("error_subgraph_indices.txt", "r") as f:
#     error_subgraph_indices = [int(index) for index in f.readlines()]

# df_valid = df.drop(error_subgraph_indices)

# # # generate a small set of questions for each type, for each type, only select 10 samples
# # small_set_df = (
# #     df_valid.groupby("type").apply(lambda x: x.sample(10)).reset_index(drop=True)
# # )
# # small_set_df.to_csv("OKG/filtered_questions_50_v3.csv")

SPARQL Template for single entity, two entities and three entities

In [5]:
# SPARQL query for single entity
single_entity_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity
WHERE {{
  {{
    dbr:{entity} ?p ?firstHopEntity.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity} AS ?entity)
  }} UNION {{
    dbr:{entity} ?p ?firstHopEntity.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity}.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity}.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity} AS ?entity)
  }}
}}
"""

# SPARQL query for two entities
two_entity_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity
WHERE {{
    {{
    dbr:{entity1} ?p ?firstHopEntity.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity1} AS ?entity)
    }} UNION {{
    dbr:{entity1} ?p ?firstHopEntity.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity1} AS ?entity)
    }} UNION {{
    ?firstHopEntity ?p dbr:{entity1}.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity1} AS ?entity)
    }} UNION {{
    ?firstHopEntity ?p dbr:{entity1}.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity1} AS ?entity)
    }} UNION {{
    dbr:{entity2} ?p ?firstHopEntity.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity2} AS ?entity)
    }} UNION {{
    dbr:{entity2} ?p ?firstHopEntity.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity2} AS ?entity)
    }} UNION {{
    ?firstHopEntity ?p dbr:{entity2}.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity2} AS ?entity)
    }} UNION {{
    ?firstHopEntity ?p dbr:{entity2}.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity2} AS ?entity)
    }}
}}
"""

# SPARQL query for three entities

three_entity_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity
WHERE {{
  {{
    dbr:{entity1} ?p ?firstHopEntity.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity1} AS ?entity)
  }} UNION {{
    dbr:{entity1} ?p ?firstHopEntity.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity1} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity1}.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity1} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity1}.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity1} AS ?entity)
  }} UNION {{
    dbr:{entity2} ?p ?firstHopEntity.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity2} AS ?entity)
  }} UNION {{
    dbr:{entity2} ?p ?firstHopEntity.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity2} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity2}.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity2} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity2}.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity2} AS ?entity)
  }} UNION {{
    dbr:{entity3} ?p ?firstHopEntity.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity3} AS ?entity)
  }} UNION {{
    dbr:{entity3} ?p ?firstHopEntity.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity3} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity3}.
    ?firstHopEntity ?p2 ?secondHopEntity.
    BIND(dbr:{entity3} AS ?entity)
  }} UNION {{
    ?firstHopEntity ?p dbr:{entity3}.
    ?secondHopEntity ?p2 ?firstHopEntity.
    BIND(dbr:{entity3} AS ?entity)
  }}
}}
"""

def run_sparql(entities):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")

    if len(entities) == 1:
        query = single_entity_query
        query = query.format(entity=entities[0])
    elif len(entities) == 2:
        query = two_entity_query
        query = query.format(entity1=entities[0], entity2=entities[1])
    elif len(entities) == 3:
        query = three_entity_query
        query = query.format(
            entity1=entities[0], entity2=entities[1], entity3=entities[2]
        )

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

save all the nodes/edges to NetworkX and then calculate the statistics of the graph

In [6]:
class PPR_Utils:
    # PPR to prune the graph
    def __init__(self):
        # define the parameters
        self.alpha = 0.85
        self.tol = 1e-6
        self.max_iter = 100
        self.threshold = 1e-5  # threshold for pruning, if the node's PPR is less than the threshold, it will be pruned

    def calculate_ppr(self, G, central_node):
        # calcultae Personalized PageRank
        personalization = {node: 0 for node in G.nodes()}
        # set the central node weight to 1
        for node in list(central_node):
            personalization[node] = 1

        ppr = nx.pagerank(
            G,
            personalization=personalization,
            alpha=self.alpha,
            tol=self.tol,
            max_iter=self.max_iter,
        )
        return ppr

    def prune_graph(self, G, central_node):
        pruned_G = nx.DiGraph()
        ppr = self.calculate_ppr(G, central_node)

        for node, score in ppr.items():
            if score >= self.threshold:
                pruned_G.add_node(node)

        for u, v, data in G.edges(data=True):
            if u in pruned_G and v in pruned_G:
                pruned_G.add_edge(u, v, **data)

        return pruned_G


def build_up_graph(rdfs):
    """_summary_

    Args:
        rdfs (dict): results from SPARQL query

    Returns:
        _type_: networkx graph
    """
    
    def extract_value(obj):
        if obj["type"] == "uri":
            return obj["value"].split("/")[-1].split("#")[-1]
        else:
            return obj["value"]
    
    # hold directed edges. self loops are allowed but multiple(parell) edges are not.
    G = nx.DiGraph()
    central_node = set()

    for result in rdfs["results"]["bindings"]:
        node = extract_value(result["entity"])
        central_node.add(node)

        first_hop_nei = extract_value(result["firstHopEntity"])
        second_hop_nei = extract_value(result["secondHopEntity"])
        r1 = extract_value(result["p"])
        r2 = extract_value(result["p2"])

        if node == first_hop_nei:
            # dbr:entity --> first_hop_nei --> second_hop_nei
            G.add_edge(node, first_hop_nei, relation=r1)
            G.add_edge(node, second_hop_nei, relation=r2)
        elif node == second_hop_nei:
            # dbr:entity -> first_hop_nei <- second_hop_nei
            G.add_edge(node, first_hop_nei, relation=r1)
            G.add_edge(second_hop_nei, first_hop_nei, relation=r2)
        elif first_hop_nei == second_hop_nei:
            # first_hop_nei -> dbr:entity -> second_hop_nei
            G.add_edge(first_hop_nei, node, relation=r1)
            G.add_edge(node, second_hop_nei, relation=r2)
        else:
            # first_hop_nei -> dbr:entity <- second_hop_nei
            G.add_edge(first_hop_nei, node, relation=r1)
            G.add_edge(second_hop_nei, first_hop_nei, relation=r2)

    return G, central_node

    
def retrieve_subgraph(index, entry_node, rerun=False):
    raw_graph_pth = f"subgraphs/raw/{index}.pkl"
    pruned_ppr_graph_pth = f"subgraphs/pruned_ppr/{index}.pkl"

    if rerun == False:
        if os.path.exists(raw_graph_pth) and os.path.exists(pruned_ppr_graph_pth):
            return index, True

    try:
        rdfs = run_sparql(entry_node)
        G, central_node = build_up_graph(rdfs)
        ppr_G = ppr.prune_graph(G, central_node)
        pickle.dump(G, open(f"subgraphs/raw/{index}.pkl", "wb"))
        pickle.dump(ppr_G, open(f"subgraphs/pruned_ppr/{index}.pkl", "wb"))
        return index, True

    except Exception as e:
        print(f"Error: {e} occurred for subgraph {index}")
        return index, False


try:
    os.mkdir("subgraphs")
    os.mkdir("subgraphs/raw")
    os.mkdir("subgraphs/pruned_ppr")
except:
    pass

error_subgraph_indices = []
ppr = PPR_Utils()

entry_nodes = []
for entities in df["dbpedia_entities_re"]:
    entry_nodes.append(list(entities.values()))
    

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = []
    for index, entry_node in enumerate(entry_nodes):
        futures.append(executor.submit(retrieve_subgraph, index, entry_node, True))
    for future in tqdm(
        as_completed(futures),
        total=len(futures),
        desc="Building subgraphs...",
        leave=True,
    ):
        index, flag = future.result()
        if flag == False:
            error_subgraph_indices.append(index)
            
len(error_subgraph_indices)
with open("subgraphs/error_subgraph_indices.txt", "w") as f:
    for index in error_subgraph_indices:
        f.write(f"{index}\n")

Building subgraphs...:   0%|          | 2/850 [00:02<14:39,  1.04s/it]

Error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 25: syntax error at '.' before '?p'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX dbr: <http://dbpedia.org/resource/>\nSELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity\nWHERE {\n    {\n    dbr:Microsoft ?p ?firstHopEntity.\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Microsoft AS ?entity)\n    } UNION {\n    dbr:Microsoft ?p ?firstHopEntity.\n    ?secondHopEntity ?p2 ?firstHopEntity.\n    BIND(dbr:Microsoft AS ?entity)\n    } UNION {\n    ?firstHopEntity ?p dbr:Microsoft.\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Microsoft AS ?entity)\n    } UNION {\n    ?firstHopEntity ?p dbr:Microsoft.\n    ?secondHopEntity ?p2 ?firstHopEntity.\n  

Building subgraphs...:   1%|          | 6/850 [00:02<03:38,  3.86it/s]

Error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 9: syntax error at ',' before '_Inc'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX dbr: <http://dbpedia.org/resource/>\nSELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity\nWHERE {\n  {\n    dbr:Tesla,_Inc. ?p ?firstHopEntity.\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Tesla,_Inc. AS ?entity)\n  } UNION {\n    dbr:Tesla,_Inc. ?p ?firstHopEntity.\n    ?secondHopEntity ?p2 ?firstHopEntity.\n    BIND(dbr:Tesla,_Inc. AS ?entity)\n  } UNION {\n    ?firstHopEntity ?p dbr:Tesla,_Inc..\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Tesla,_Inc. AS ?entity)\n  } UNION {\n    ?firstHopEntity ?p dbr:Tesla,_Inc..\n    ?secondHopEntity ?p2 ?firstHopEnti

Building subgraphs...:   2%|▏         | 19/850 [00:39<16:36,  1.20s/it]

...

Building subgraphs...:  91%|█████████▏| 776/850 [06:12<00:13,  5.29it/s]

Error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 25: syntax error at 'search_engine'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX dbr: <http://dbpedia.org/resource/>\nSELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity\nWHERE {\n    {\n    dbr:Google_Search ?p ?firstHopEntity.\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Google_Search AS ?entity)\n    } UNION {\n    dbr:Google_Search ?p ?firstHopEntity.\n    ?secondHopEntity ?p2 ?firstHopEntity.\n    BIND(dbr:Google_Search AS ?entity)\n    } UNION {\n    ?firstHopEntity ?p dbr:Google_Search.\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Google_Search AS ?entity)\n    } UNION {\n    ?firstHopEntity ?p dbr:Google_Search.\n    ?secondHopEnt

Building subgraphs...:  92%|█████████▏| 778/850 [06:14<00:41,  1.73it/s]

Error: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 9: syntax error at '.' before '?p'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX dbr: <http://dbpedia.org/resource/>\nSELECT DISTINCT ?entity ?p ?firstHopEntity ?p2 ?secondHopEntity\nWHERE {\n  {\n    dbr:Martin_Luther_King_Jr. ?p ?firstHopEntity.\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Martin_Luther_King_Jr. AS ?entity)\n  } UNION {\n    dbr:Martin_Luther_King_Jr. ?p ?firstHopEntity.\n    ?secondHopEntity ?p2 ?firstHopEntity.\n    BIND(dbr:Martin_Luther_King_Jr. AS ?entity)\n  } UNION {\n    ?firstHopEntity ?p dbr:Martin_Luther_King_Jr..\n    ?firstHopEntity ?p2 ?secondHopEntity.\n    BIND(dbr:Martin_Luther_King_Jr. AS ?entity)\n  } UNION {\n    ?firstHopEn

Building subgraphs...: 100%|██████████| 850/850 [06:45<00:00,  2.10it/s]


Calculate the statistics of the builded graph

In [61]:
import tiktoken


def calculate_statistics(G: nx.DiGraph):
    """_summary_

    Args:
       G (nx.DiGraph): _description_
    """

    # statistics
    # print("Graph Statistics:")

    # print(f"Number of nodes: {G.number_of_nodes()}")
    # print(f"Number of edges: {G.number_of_edges()}")

    tokenizer = tiktoken.encoding_for_model("gpt-4o")

    edge_list = nx.generate_edgelist(G, data=True)
    tokens = len(tokenizer.encode("".join(edge_list)))

    number_of_nodes = G.number_of_nodes()
    number_of_edges = G.number_of_edges()

    # Degree statistics
    in_degrees = [d for n, d in G.in_degree()]
    out_degrees = [d for n, d in G.out_degree()]
    total_degrees = [d for n, d in G.degree()]

    # print(f"Average in-degree: {sum(in_degrees) / len(in_degrees):.2f}")
    # print(f"Average out-degree: {sum(out_degrees) / len(out_degrees):.2f}")
    # print(f"Average total degree: {sum(total_degrees) / len(total_degrees):.2f}")
    avg_in_degree = sum(in_degrees) / len(in_degrees)
    avg_out_degree = sum(out_degrees) / len(out_degrees)
    avg_total_degree = sum(total_degrees) / len(total_degrees)

    # # Most common relations
    # relations = [data['label'] for u, v, data in G.edges(data=True)]
    # common_relations = Counter(relations).most_common(5)
    # print("\nTop 5 most common relations:")
    # for relation, count in common_relations:
    #     print(f"{relation}: {count}")

    # # Number of connected components (for undirected version of the graph)
    # undirected_G = G.to_undirected()
    # num_components = nx.number_connected_components(undirected_G)
    # print(f"\nNumber of connected components: {num_components}")

    # # Largest connected component
    # largest_cc = max(nx.connected_components(undirected_G), key=len)
    # print(f"Size of the largest connected component: {len(largest_cc)}")

    # # Check if the graph is a DAG (Directed Acyclic Graph)
    # is_dag = nx.is_directed_acyclic_graph(G)
    # print(f"\nIs the graph a DAG? {is_dag}")
    # print(f"Is the graph strongly connected? {nx.is_strongly_connected(G)}")
    # print(f"Is the graph weakly connected? {nx.is_weakly_connected(G)}")

    # # Calculate the diameter of the largest connected component
    # largest_cc_subgraph = undirected_G.subgraph(largest_cc)
    # diameter = nx.diameter(largest_cc_subgraph)
    # print(f"Diameter of the largest connected component: {diameter}")

    # # Calculate the centrality of the top 5 nodes
    # print("\nCentrality Measures (for top 5 nodes):")
    # in_degree_centrality = nx.in_degree_centrality(G)
    # print("Top 5 nodes by In-Degree Centrality:")
    # for node, centrality in sorted(
    #     in_degree_centrality.items(), key=lambda x: x[1], reverse=True
    # )[:5]:
    #     print(f"{node}: {centrality:.4f}")

    # out_degree_centrality = nx.out_degree_centrality(G)
    # print("\nTop 5 nodes by Out-Degree Centrality:")
    # for node, centrality in sorted(
    #     out_degree_centrality.items(), key=lambda x: x[1], reverse=True
    # )[:5]:
    #     print(f"{node}: {centrality:.4f}")

    # # Betweenness Centrality (can be slow for large graphs)
    # betweenness_centrality = nx.betweenness_centrality(G)
    # print("\nTop 5 nodes by Betweenness Centrality:")
    # for node, centrality in sorted(
    #     betweenness_centrality.items(), key=lambda x: x[1], reverse=True
    # )[:5]:
    #     print(f"{node}: {centrality:.4f}")

    # Clustering Coefficient
    clustering_coefficient = nx.average_clustering(G)
    # print(f"\nAverage Clustering Coefficient: {clustering_coefficient:.4f}")

    # Shortest Paths
    # print("\nShortest Path Statistics:")
    # shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
    # path_lengths = [
    #     length for paths in shortest_paths.values() for length in paths.values()
    # ]
    # print(f"Average Shortest Path Length: {np.mean(path_lengths):.2f}")
    # print(f"Maximum Shortest Path Length (Diameter): {max(path_lengths)}")

    # Density
    density = nx.density(G)
    # print(f"\nGraph Density: {density:.4f}")

    return (
        tokens,
        number_of_nodes,
        number_of_edges,
        avg_in_degree,
        avg_out_degree,
        avg_total_degree,
        clustering_coefficient,
        density,
    )

Calculate the overall statistics

In [10]:
with open("subgraphs/error_subgraph_indices.txt", "r") as f:
    error_subgraph_indices = [int(index) for index in f.readlines()]

df = df.drop(error_subgraph_indices)

In [29]:
def avg_statistics_of_G(Graphs):
    value = [0 for _ in range(8)]
    for G in Graphs:
        # calculate the average tokens, average nodes, average edges, average in-degree, out-degree, total-degree, average clustering coefficency, average graph density
        stats = calculate_statistics(G)
        value = [v + s for v, s in zip(value, stats)]

    avg_value = [v / len(df) for v in value]

    print(f"Average Tokens: {avg_value[0]:.2f}")
    print(f"Average Number of Nodes: {avg_value[1]:.2f}")
    print(f"Average Number of Edges: {avg_value[2]:.2f}")
    print(f"Average In-Degree: {avg_value[3]:.2f}")
    print(f"Average Out-Degree: {avg_value[4]:.2f}")
    print(f"Average Total Degree: {avg_value[5]:.2f}")
    print(f"Average Clustering Coefficient: {avg_value[6]:.2f}")
    print(f"Average Graph Density: {avg_value[7]:.2f}")

    return avg_value

In [11]:
import pickle
import os


def load_all_graphs(raw_graph_dir):
    """
    Loads all pickle graph files from the specified directory.

    Parameters:
    - raw_graph_dir (str): Path to the directory containing raw graph pickle files.

    Returns:
    - graphs (dict): A dictionary where keys are filenames and values are the loaded graph objects.
    """
    graphs = []

    # Check if the directory exists
    if not os.path.isdir(raw_graph_dir):
        raise FileNotFoundError(f"The directory '{raw_graph_dir}' does not exist.")

    # Iterate through all files in the directory
    for filename in os.listdir(raw_graph_dir):
        filepath = os.path.join(raw_graph_dir, filename)

        # Check if it's a file and has a .pkl extension
        if os.path.isfile(filepath) and filename.endswith('.pkl'):
            try:
                with open(filepath, 'rb') as file:
                    graph = pickle.load(file)
                    
                    # Optional: If using NetworkX, verify it's a graph
                    if isinstance(graph, nx.Graph):
                        # print(f"Loaded NetworkX graph '{filename}' with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
                        print(f"Loaded NetworkX graph '{filename}'.")
                    else:
                        print(f"Loaded object from '{filename}'. It may not be a NetworkX graph.")
                    
                    graphs.append(graph)
            except Exception as e:
                print(f"Failed to load '{filename}': {e}")

    return graphs

raw_graphs = load_all_graphs("subgraphs/raw/")
pruned_ppr_graphs = load_all_graphs("subgraphs/pruned_ppr/")

Loaded NetworkX graph '438.pkl'.
Loaded NetworkX graph '68.pkl'.
Loaded NetworkX graph '342.pkl'.
Loaded NetworkX graph '787.pkl'.
Loaded NetworkX graph '327.pkl'.
Loaded NetworkX graph '323.pkl'.
Loaded NetworkX graph '774.pkl'.
Loaded NetworkX graph '486.pkl'.
Loaded NetworkX graph '34.pkl'.
Loaded NetworkX graph '103.pkl'.
Loaded NetworkX graph '398.pkl'.
Loaded NetworkX graph '701.pkl'.
Loaded NetworkX graph '754.pkl'.
Loaded NetworkX graph '295.pkl'.
Loaded NetworkX graph '43.pkl'.
Loaded NetworkX graph '463.pkl'.
Loaded NetworkX graph '843.pkl'.
Loaded NetworkX graph '655.pkl'.
Loaded NetworkX graph '639.pkl'.
Loaded NetworkX graph '40.pkl'.
Loaded NetworkX graph '500.pkl'.
Loaded NetworkX graph '276.pkl'.
Loaded NetworkX graph '461.pkl'.
Loaded NetworkX graph '477.pkl'.
Loaded NetworkX graph '839.pkl'.
Loaded NetworkX graph '67.pkl'.
Loaded NetworkX graph '330.pkl'.
Loaded NetworkX graph '790.pkl'.
Loaded NetworkX graph '271.pkl'.
Loaded NetworkX graph '161.pkl'.
Loaded NetworkX

In [12]:
# for u, v, d in raw_graphs[0].edges(data=True):
#     print(u, d["relation"], v)

raw_graphs[0].edges(data=True)

OutEdgeDataView([('Thing', 'Blockchain', {'relation': 'type'}), ('Thing', 'PricewaterhouseCoopers', {'relation': 'type'}), ('Thing', 'Privacy_and_blockchain', {'relation': 'type'}), ('Thing', 'Pseudonym', {'relation': 'type'}), ('Thing', 'Saraju_Mohanty', {'relation': 'type'}), ('Thing', 'Satoshi_Nakamoto', {'relation': 'type'}), ('Thing', 'Metaverse', {'relation': 'type'}), ('Thing', 'Moni_Naor', {'relation': 'type'}), ('Thing', 'Non-fungible_token', {'relation': 'type'}), ('Thing', 'Moral_hazard', {'relation': 'type'}), ('Thing', 'Berenberg_Bank', {'relation': 'type'}), ('Thing', 'Bill_Gates', {'relation': 'type'}), ('Thing', 'Bitcoin', {'relation': 'type'}), ('Thing', 'Dave_Bayer', {'relation': 'type'}), ('Thing', 'David_A._Marcus', {'relation': 'type'}), ('Thing', 'David_Chaum', {'relation': 'type'}), ('Thing', 'Ubisoft', {'relation': 'type'}), ('Thing', 'United_Nations_Economic_Commission_for_Europe', {'relation': 'type'}), ('Thing', 'United_States_dollar', {'relation': 'type'}), 

In [16]:
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from typing import Callable
import random
import os

# Set random seeds for reproducibility
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# ============================
# 1. Data Preparation
# ============================

class KGLinkPredictionDataset(Dataset):
    """
    Dataset for Link Prediction in Knowledge Graphs.
    Each sample consists of a triple represented as a string and a label indicating its validity.
    """
    def __init__(self, triples, labels, tokenizer, max_length=512):
        self.triples = triples
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.triples)
    
    def __getitem__(self, idx):
        triple = self.triples[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            triple,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),  # Tensor of shape (max_length)
            'attention_mask': encoding['attention_mask'].flatten(),  # Tensor of shape (max_length)
            'labels': torch.tensor(label, dtype=torch.long)
        }

def prepare_data(G: nx.DiGraph, tokenizer, negative_ratio=1.0, max_samples=None):
    """
    Prepare positive and negative samples for link prediction.

    Parameters:
    - G (nx.DiGraph): The raw knowledge graph.
    - tokenizer: The tokenizer to convert triples into text.
    - negative_ratio (float): Ratio of negative to positive samples.
    - max_samples (int or None): Maximum number of positive samples to use.

    Returns:
    - triples (List[str]): List of triple strings.
    - labels (List[int]): Corresponding labels (1 for positive, 0 for negative).
    """
    # Extract all positive triples
    positive_triples = [(u, d['relation'], v) for u, v, d in G.edges(data=True)]
    
    if max_samples:
        positive_triples = positive_triples[:max_samples]
    
    triples = []
    labels = []
    
    # Positive samples
    for triple in positive_triples:
        e1, r, e2 = triple
        triple_str = f"{e1} {r} {e2}"
        triples.append(triple_str)
        labels.append(1)
    
    # Negative samples
    num_negative = int(len(positive_triples) * negative_ratio)
    entities = list(G.nodes())
    relations = list(set(nx.get_edge_attributes(G, 'relation').values()))
    
    negative_triples = set()
    while len(negative_triples) < num_negative:
        e1, r, e2 = random.choice(entities), random.choice(relations), random.choice(entities)
        if not G.has_edge(e1, e2):
            negative_triples.add((e1, r, e2))
    
    for triple in negative_triples:
        e1, r, e2 = triple
        triple_str = f"{e1} {r} {e2}"
        triples.append(triple_str)
        labels.append(0)
    
    return triples, labels

# ============================
# 2. Model Training
# ============================

def train_model(G: nx.DiGraph, model_save_path: str = "link_prediction_model"):
    """
    Train a DistilBERT model for link prediction on the given knowledge graph.

    Parameters:
    - G (nx.DiGraph): The raw knowledge graph.
    - model_save_path (str): Directory to save the trained model.

    Returns:
    - tokenizer: The tokenizer used.
    - model: The trained model.
    """
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
    
    # Prepare dataset
    triples, labels = prepare_data(G, tokenizer, negative_ratio=1.0)
    
    # Split into train and validation
    triples_train, triples_val, labels_train, labels_val = train_test_split(
        triples, labels, test_size=0.2, random_state=42, stratify=labels
    )
    
    train_dataset = KGLinkPredictionDataset(triples_train, labels_train, tokenizer)
    val_dataset = KGLinkPredictionDataset(triples_val, labels_val, tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    
    # Move model to device
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    
    # Define optimizer and loss
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.CrossEntropyLoss()
    
    # Training loop
    epochs = 3
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_batch = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_batch)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels_batch = batch['labels'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                correct += (predictions == labels_batch).sum().item()
                total += labels_batch.size(0)
        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy:.4f}")
    
    # Save the model
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print(f"Model saved to {model_save_path}")
    
    return tokenizer, model

# ============================
# 3. Scoring Function s_G
# ============================

class TripleScorer:
    """
    Scoring function s_G that uses a trained language model to score triples.
    """
    def __init__(self, model_path: str = "link_prediction_model"):
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = DistilBertForSequenceClassification.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()
    
    def score(self, e1: str, r: str, e2: str) -> float:
        """
        Score a triple (e1, r, e2) using the trained model.

        Parameters:
        - e1 (str): Head entity.
        - r (str): Relation.
        - e2 (str): Tail entity.

        Returns:
        - float: Probability score in [0, 1].
        """
        triple_str = f"{e1} {r} {e2}"
        encoding = self.tokenizer.encode_plus(
            triple_str,
            add_special_tokens=True,
            max_length=50,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=-1)
            # Assuming label 1 is the positive class
            prob_positive = probabilities[:, 1].item()
        return prob_positive

In [22]:
import networkx as nx
import numpy as np
from typing import Callable
import copy

def compute_ATS(
    G: nx.DiGraph,
    G_prime: nx.DiGraph,
    s_G: Callable[[str, str, str], float]
) -> float:
    """
    Compute Aggregated Triple Score (ATS) between two Knowledge Graphs.

    Parameters:
    - G (nx.DiGraph): The raw knowledge graph.
    - G_prime (nx.DiGraph): The perturbed knowledge graph.
    - s_G (Callable): A scoring function that takes (e1, r, e2) and returns a score in [0, 1].

    Returns:
    - float: The ATS score, a value between 0 and 1.
    """
    if len(G_prime.edges) == 0:
        return 0.0  # Avoid division by zero

    scores = []
    for u, v, data in G_prime.edges(data=True):
        r = data.get('relation')
        e1, e2 = u, v
        score = s_G(e1, r, e2)
        scores.append(score)
    
    ats = np.mean(scores)
    return ats

def compute_SC2D(G: nx.DiGraph, G_prime: nx.DiGraph) -> float:
    """
    Compute Similarity in Clustering Coefficient Distribution (SC2D) between two KGs.

    Parameters:
    - G (nx.DiGraph): The raw knowledge graph.
    - G_prime (nx.DiGraph): The perturbed knowledge graph.

    Returns:
    - float: The SC2D score, a value between 0 and 1.
    """
    def get_mean_clustering(graph: nx.DiGraph) -> np.ndarray:
        relations = set(nx.get_edge_attributes(graph, 'relation').values())
        if not relations:
            return np.zeros(len(graph.nodes()))
        
        c_sum = np.zeros(len(graph.nodes()))
        node_list = list(graph.nodes())
        node_index = {node: idx for idx, node in enumerate(node_list)}
        
        for r in relations:
            # Extract subgraph for relation r
            edges_r = [(u, v) for u, v, d in graph.edges(data=True) if d.get('relation') == r]
            subgraph = nx.DiGraph(edges_r)
            undirected_subgraph = subgraph.to_undirected()
            
            # Compute clustering coefficients
            clustering = nx.clustering(undirected_subgraph)
            c_vector = np.zeros(len(node_list))
            for node, c in clustering.items():
                idx = node_index[node]
                c_vector[idx] = c
            c_sum += c_vector
        
        mean_clustering = c_sum / len(relations)
        return mean_clustering

    # Ensure both graphs have the same node ordering
    all_nodes = sorted(set(G.nodes()).union(set(G_prime.nodes())))
    G = copy.deepcopy(G)
    G_prime = copy.deepcopy(G_prime)
    G.add_nodes_from(all_nodes)
    G_prime.add_nodes_from(all_nodes)

    c_o = get_mean_clustering(G)
    c_p = get_mean_clustering(G_prime)
    
    norm_diff = np.linalg.norm(c_o - c_p)
    sc2d = 1 - (norm_diff / (norm_diff + 1))
    return sc2d

def compute_SD2(G: nx.DiGraph, G_prime: nx.DiGraph) -> float:
    """
    Compute Similarity in Degree Distribution (SD2) between two KGs.

    Parameters:
    - G (nx.DiGraph): The raw knowledge graph.
    - G_prime (nx.DiGraph): The perturbed knowledge graph.

    Returns:
    - float: The SD2 score, a value between 0 and 1.
    """
    def get_mean_degree(graph: nx.DiGraph) -> np.ndarray:
        relations = set(nx.get_edge_attributes(graph, 'relation').values())
        if not relations:
            return np.zeros(len(graph.nodes()))
        
        d_sum = np.zeros(len(graph.nodes()))
        node_list = list(graph.nodes())
        node_index = {node: idx for idx, node in enumerate(node_list)}
        
        for r in relations:
            # Extract subgraph for relation r
            edges_r = [(u, v) for u, v, d in graph.edges(data=True) if d.get('relation') == r]
            subgraph = nx.DiGraph(edges_r)
            
            # Compute degrees (in-degree + out-degree)
            degrees = dict(subgraph.degree())
            d_vector = np.zeros(len(node_list))
            for node, deg in degrees.items():
                idx = node_index[node]
                d_vector[idx] = deg
            d_sum += d_vector
        
        mean_degree = d_sum / len(relations)
        return mean_degree

    # Ensure both graphs have the same node ordering
    all_nodes = sorted(set(G.nodes()).union(set(G_prime.nodes())))
    G = copy.deepcopy(G)
    G_prime = copy.deepcopy(G_prime)
    G.add_nodes_from(all_nodes)
    G_prime.add_nodes_from(all_nodes)

    d_o = get_mean_degree(G)
    d_p = get_mean_degree(G_prime)
    
    norm_diff = np.linalg.norm(d_o - d_p)
    sd2 = 1 - (norm_diff / (norm_diff + 1))
    return sd2

def compute_all_metrics(
    G: nx.DiGraph,
    G_prime: nx.DiGraph,
    s_G: Callable[[str, str, str], float]
) -> dict:
    """
    Compute all three metrics: ATS, SC2D, and SD2.

    Parameters:
    - G (nx.DiGraph): The raw knowledge graph.
    - G_prime (nx.DiGraph): The perturbed knowledge graph.
    - s_G (Callable): The ATS scoring function.

    Returns:
    - dict: A dictionary containing the three metrics.
    """
    ats = compute_ATS(G, G_prime, s_G)
    sc2d = compute_SC2D(G, G_prime)
    sd2 = compute_SD2(G, G_prime)
    
    return {
        'ATS': ats,
        'SC2D': sc2d,
        'SD2': sd2
    }

In [24]:
import networkx as nx
import numpy as np
import random
from typing import Callable, List

def perturb_relation_swapping(
    G: nx.DiGraph,
    perturbation_level: float,
    seed: int = 42
) -> nx.DiGraph:
    """
    Perform Relation Swapping (RS) on the knowledge graph.

    Parameters:
    - G (nx.DiGraph): The original knowledge graph.
    - perturbation_level (float): Percentage of edges to perturb (0 < p < 1).
    - seed (int): Random seed for reproducibility.

    Returns:
    - G_perturbed (nx.DiGraph): The perturbed knowledge graph.
    """
    random.seed(seed)
    G_perturbed = copy.deepcopy(G)
    total_edges = G_perturbed.number_of_edges()
    num_swaps = int((perturbation_level * total_edges) / 2)

    edges = list(G_perturbed.edges(data=True))
    if len(edges) < 2:
        print("Not enough edges to perform Relation Swapping.")
        return G_perturbed

    for _ in range(num_swaps):
        # Randomly select two distinct edges
        edge1, edge2 = random.sample(edges, 2)
        u1, v1, data1 = edge1
        u2, v2, data2 = edge2

        # Swap their relations
        G_perturbed[u1][v1]['relation'], G_perturbed[u2][v2]['relation'] = (
            G_perturbed[u2][v2]['relation'],
            G_perturbed[u1][v1]['relation']
        )

    print(f"Relation Swapping: Swapped relations of {num_swaps * 2} edges.")
    return G_perturbed

def perturb_relation_replacement(
    G: nx.DiGraph,
    perturbation_level: float,
    s_G: Callable[[str, str, str], float],
    relations: List[str],
    seed: int = 42
) -> nx.DiGraph:
    """
    Perform Relation Replacement (RR) on the knowledge graph.

    Parameters:
    - G (nx.DiGraph): The original knowledge graph.
    - perturbation_level (float): Percentage of edges to perturb (0 < p < 1).
    - s_G (Callable): Scoring function s_G(e1, r, e2).
    - relations (List[str]): List of all possible relations.
    - seed (int): Random seed for reproducibility.

    Returns:
    - G_perturbed (nx.DiGraph): The perturbed knowledge graph.
    """
    random.seed(seed)
    G_perturbed = copy.deepcopy(G)
    total_edges = G_perturbed.number_of_edges()
    num_replacements = int(perturbation_level * total_edges)

    edges = list(G_perturbed.edges(data=True))
    if not edges:
        print("No edges available for Relation Replacement.")
        return G_perturbed

    for _ in range(num_replacements):
        # Randomly select an edge
        edge = random.choice(edges)
        u, v, data = edge
        original_relation = data['relation']

        # Find the relation that minimizes the semantic similarity
        # Exclude the original relation to ensure a change
        candidate_relations = [r for r in relations if r != original_relation]
        if not candidate_relations:
            print("No alternative relations available for replacement.")
            continue

        # Compute scores for all candidate relations
        scores = [s_G(u, r, v) for r in candidate_relations]
        if not scores:
            print("Scoring function returned empty scores.")
            continue

        # Select the relation with the minimum score
        min_index = np.argmin(scores)
        new_relation = candidate_relations[min_index]

        # Replace the relation
        G_perturbed[u][v]['relation'] = new_relation

    print(f"Relation Replacement: Replaced relations of {num_replacements} edges.")
    return G_perturbed

def perturb_edge_rewiring(
    G: nx.DiGraph,
    perturbation_level: float,
    seed: int = 42
) -> nx.DiGraph:
    """
    Perform Edge Rewiring (ER) on the knowledge graph.

    Parameters:
    - G (nx.DiGraph): The original knowledge graph.
    - perturbation_level (float): Percentage of edges to perturb (0 < p < 1).
    - seed (int): Random seed for reproducibility.

    Returns:
    - G_perturbed (nx.DiGraph): The perturbed knowledge graph.
    """
    random.seed(seed)
    G_perturbed = copy.deepcopy(G)
    total_edges = G_perturbed.number_of_edges()
    num_rewirings = int(perturbation_level * total_edges)

    edges = list(G_perturbed.edges(data=True))
    entities = list(G_perturbed.nodes())
    if not edges or not entities:
        print("No edges or entities available for Edge Rewiring.")
        return G_perturbed

    for _ in range(num_rewirings):
        # Randomly select an edge
        edge = random.choice(edges)
        u, _, data = edge
        original_relation = data['relation']

        # Determine 1-hop neighborhood of u
        neighborhood = set(G_perturbed.successors(u)) | set(G_perturbed.predecessors(u))
        neighborhood.add(u)  # Exclude self

        # Possible new entities to connect to
        possible_entities = list(set(entities) - neighborhood)
        if not possible_entities:
            print(f"No available entities to rewire from entity '{u}'.")
            continue

        # Select a new entity e3
        e3 = random.choice(possible_entities)

        # Replace the tail entity
        try:
            G_perturbed.remove_edge(u, edge[1])
            G_perturbed.add_edge(u, e3, relation=original_relation)
        except nx.NetworkXError:
            print(f"Failed to rewire edge from entity '{u}' to '{e3}'.")

    print(f"Edge Rewiring: Rewired {num_rewirings} edges.")
    return G_perturbed

def perturb_edge_deletion(
    G: nx.DiGraph,
    perturbation_level: float,
    seed: int = 42
) -> nx.DiGraph:
    """
    Perform Edge Deletion (ED) on the knowledge graph.

    Parameters:
    - G (nx.DiGraph): The original knowledge graph.
    - perturbation_level (float): Percentage of edges to perturb (0 < p < 1).
    - seed (int): Random seed for reproducibility.

    Returns:
    - G_perturbed (nx.DiGraph): The perturbed knowledge graph.
    """
    random.seed(seed)
    G_perturbed = copy.deepcopy(G)
    total_edges = G_perturbed.number_of_edges()
    num_deletions = int(perturbation_level * total_edges)

    edges = list(G_perturbed.edges())
    if not edges:
        print("No edges available for Edge Deletion.")
        return G_perturbed

    edges_to_delete = random.sample(edges, min(num_deletions, len(edges)))

    G_perturbed.remove_edges_from(edges_to_delete)

    print(f"Edge Deletion: Deleted {len(edges_to_delete)} edges.")
    return G_perturbed

def apply_perturbation(
    G: nx.DiGraph,
    method: str,
    perturbation_level: float,
    s_G: Callable[[str, str, str], float] = None,
    relations: List[str] = None,
    seed: int = 42
) -> nx.DiGraph:
    """
    Apply a specified perturbation method to the knowledge graph.

    Parameters:
    - G (nx.DiGraph): The original knowledge graph.
    - method (str): Perturbation method to apply ('RS', 'RR', 'ER', 'ED').
    - perturbation_level (float): Percentage of edges to perturb (0 < p < 1).
    - s_G (Callable, optional): Scoring function for RR. Required if method is 'RR'.
    - relations (List[str], optional): List of all possible relations. Required if method is 'RR'.
    - seed (int): Random seed for reproducibility.

    Returns:
    - G_perturbed (nx.DiGraph): The perturbed knowledge graph.
    """
    method = method.upper()
    if not (0 < perturbation_level <= 1):
        raise ValueError("Perturbation level must be between 0 and 1.")

    if method == 'RS':
        if G.number_of_edges() < 2:
            print("Not enough edges to perform Relation Swapping.")
            return copy.deepcopy(G)
        return perturb_relation_swapping(G, perturbation_level, seed)

    elif method == 'RR':
        if s_G is None or relations is None:
            raise ValueError("Scoring function 's_G' and 'relations' list must be provided for Relation Replacement.")
        if G.number_of_edges() == 0:
            print("No edges available for Relation Replacement.")
            return copy.deepcopy(G)
        return perturb_relation_replacement(G, perturbation_level, s_G, relations, seed)

    elif method == 'ER':
        if G.number_of_edges() == 0 or G.number_of_nodes() < 2:
            print("Not enough edges or entities to perform Edge Rewiring.")
            return copy.deepcopy(G)
        return perturb_edge_rewiring(G, perturbation_level, seed)

    elif method == 'ED':
        if G.number_of_edges() == 0:
            print("No edges available for Edge Deletion.")
            return copy.deepcopy(G)
        return perturb_edge_deletion(G, perturbation_level, seed)

    else:
        raise ValueError(f"Unknown perturbation method '{method}'. Choose from 'RS', 'RR', 'ER', 'ED'.")

In [25]:
if __name__ == "__main__":

    # # Create raw knowledge graph G
    # G = nx.DiGraph()
    # G.add_edge('Alice', 'Bob', relation='knows')
    # G.add_edge('Bob', 'Chocolate', relation='likes')
    # G.add_edge('Alice', 'Apples', relation='likes')
    # G.add_edge('Charlie', 'Alice', relation='knows')

    # # # Create perturbed knowledge graph G_prime
    # # G_prime = nx.DiGraph()
    # # G_prime.add_edge('Alice', 'Bob', relation='knows')
    # # G_prime.add_edge('Bob', 'Vanilla', relation='likes')  # Changed from Chocolate to Vanilla
    # # G_prime.add_edge('Alice', 'Apples', relation='likes')
    # # G_prime.add_edge('Charlie', 'Alice', relation='knows')
    # # G_prime.add_edge('Charlie', 'Bananas', relation='likes')  # New edge
    
    G_all = nx.compose_all(pruned_ppr_graphs)
    
    # ----------------------------
    # Train the Link Prediction Model
    # ----------------------------
    if not os.path.exists("/data/yuansui/link_prediction_model"):
        print("Training the link prediction model...")
        tokenizer, model = train_model(G_all, model_save_path="/data/yuansui/link_prediction_model")
    
    # ----------------------------
    # Initialize the Scoring Function
    # ----------------------------
    print("Initializing the scoring function...")
    scorer = TripleScorer(model_path="/data/yuansui/link_prediction_model")
    
    all_relations = list(set(nx.get_edge_attributes(G_all, 'relation').values()))
    print(f"All Relations: {all_relations}")
    
    perturbation_level = 0.3  # 10%
    metrics_dict = {
        'Method': [],
        'ATS': [],
        'SC2D': [],
        'SD2': []
    }
    
    # ----------------------------
    # Apply Perturbation Methods
    # ----------------------------
    
    for G in tqdm(pruned_ppr_graphs, desc="Perturbing Graphs", leave=True):
        
        # 0. RS, RR, ER, ED
        G_RSs, G_RRs, G_ERs, G_EDs = [], [], [], []

        
        # 1. Relation Swapping (RS)
        print("\n--- Applying Relation Swapping (RS) ---")
        G_RS = apply_perturbation(
            G=G,
            method='RS',
            perturbation_level=perturbation_level,
            seed=42
        )
        G_RSs.append(G_RS)
        metrics_RS = compute_all_metrics(G, G_RS, scorer.score)
        metrics_dict['Method'].append('RS')
        metrics_dict['ATS'].append(metrics_RS['ATS'])
        metrics_dict['SC2D'].append(metrics_RS['SC2D'])
        metrics_dict['SD2'].append(metrics_RS['SD2'])
        # print(f"RS Perturbed Graph Edges: {list(G_RS.edges(data=True))}")

        # 2. Relation Replacement (RR)
        print("\n--- Applying Relation Replacement (RR) ---")
        G_RR = apply_perturbation(
            G=G,
            method='RR',
            perturbation_level=perturbation_level,
            s_G=scorer.score,
            relations=all_relations,
            seed=42
        )
        G_RRs.append(G_RR)
        metrics_RR = compute_all_metrics(G, G_RR, scorer.score)
        metrics_dict['Method'].append('RR')
        metrics_dict['ATS'].append(metrics_RR['ATS'])
        metrics_dict['SC2D'].append(metrics_RR['SC2D'])
        metrics_dict['SD2'].append(metrics_RR['SD2'])
        # print(f"RR Perturbed Graph Edges: {list(G_RR.edges(data=True))}")

        # 3. Edge Rewiring (ER)
        print("\n--- Applying Edge Rewiring (ER) ---")
        G_ER = apply_perturbation(
            G=G,
            method='ER',
            perturbation_level=perturbation_level,
            seed=42
        )
        G_ERs.append(G_ER)
        metrics_ER = compute_all_metrics(G, G_ER, scorer.score)
        metrics_dict['Method'].append('ER')
        metrics_dict['ATS'].append(metrics_ER['ATS'])
        metrics_dict['SC2D'].append(metrics_ER['SC2D'])
        metrics_dict['SD2'].append(metrics_ER['SD2'])
        
        # print(f"ER Perturbed Graph Edges: {list(G_ER.edges(data=True))}")

        # 4. Edge Deletion (ED)
        print("\n--- Applying Edge Deletion (ED) ---")
        G_ED = apply_perturbation(
            G=G,
            method='ED',
            perturbation_level=perturbation_level,
            seed=42
        )
        G_EDs.append(G_ED)
        metrics_ED = compute_all_metrics(G, G_ED, scorer.score)
        metrics_dict['Method'].append('ED')
        metrics_dict['ATS'].append(metrics_ED['ATS'])
        metrics_dict['SC2D'].append(metrics_ED['SC2D'])
        metrics_dict['SD2'].append(metrics_ED['SD2'])
        # print(f"ED Perturbed Graph Edges: {list(G_ED.edges(data=True))}")
    
    # ----------------------------
    # Compute Metrics
    # ----------------------------
    metrics_df = pd.DataFrame(metrics_dict)
    average_metrics = metrics_df.groupby('Method').mean().reset_index()
    print("\n===== Average Metrics per Perturbation Method =====")
    print(metrics_df.head())
    print(f"{average_metrics:.4f}")

Initializing the scoring function...
All Relations: ['cause', 'otop', 'locationCity', 'predecessor', 'time', 'languages', 'sameAs', 'outflow', 'leaderName', 'spokenIn', 'comment', 'restingPlace', 'ownership', 'isPrimaryTopicOf', 'issuanceSchedule', 'treatment', 'subdivisionType', 'territory', 'jurisdiction', 'crewMember', 'notableWork', 'region', 'characters', 'state', 'subject', 'mother', 'date', 'exchangeRate', 'wikiPageLength', 'q', 'mint', 'usingCountries', 'prefLabel', 'notableIdea', 'administrativeCenter', 'succeededBy', 'operatingSystem', 'series', 'first', 'wikiPageRevisionID', 'mainInterest', 'starring', 'computingPlatform', 'license', 'place', 'product', 'wikiPageID', 'role', 'programmingLanguages', 'totalWidth', 'precededBy', 'blank6InfoSec', 'depiction', 'nationality', 'party', 'accessdate', 'family', 'dynasty', 'keyPerson', 'burialPlace', 'after', 'allegiance', 'primaryTopic', 'presidents', 'previousMission', 'implementations', 'thumbnail', 'video', 'sovereigntyType', 'eve

Perturbing Graphs:   0%|          | 0/792 [00:00<?, ?it/s]


--- Applying Relation Swapping (RS) ---
Relation Swapping: Swapped relations of 36 edges.

--- Applying Relation Replacement (RR) ---
Relation Replacement: Replaced relations of 36 edges.

--- Applying Edge Rewiring (ER) ---
No available entities to rewire from entity 'Blockchain'.
No available entities to rewire from entity 'Blockchain'.
Failed to rewire edge from entity 'Thing' to 'Saraju_Mohanty'.
No available entities to rewire from entity 'Blockchain'.
Failed to rewire edge from entity 'Thing' to 'Saraju_Mohanty'.
No available entities to rewire from entity 'Blockchain'.
No available entities to rewire from entity 'Blockchain'.
No available entities to rewire from entity 'Blockchain'.
Failed to rewire edge from entity 'Best-effort_delivery' to 'Moral_hazard'.
No available entities to rewire from entity 'Blockchain'.
Failed to rewire edge from entity 'United_States_dollar' to 'University_of_Cambridge'.
Edge Rewiring: Rewired 36 edges.

--- Applying Edge Deletion (ED) ---
Edge Dele

Perturbing Graphs:   0%|          | 1/792 [01:00<13:11:36, 60.05s/it]


--- Applying Relation Swapping (RS) ---
Relation Swapping: Swapped relations of 44 edges.

--- Applying Relation Replacement (RR) ---
Relation Replacement: Replaced relations of 45 edges.

--- Applying Edge Rewiring (ER) ---
No available entities to rewire from entity 'Mahatma_Gandhi'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
Failed to rewire edge from entity 'Battle_of_Spion_Kop' to 'Bihar'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
Failed to rewire edge from entity 'Thing' to 'Bollywood'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
Failed to rewire edge from entity 'Person' to 'Pranami'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
No available entities to rewire from entity 'Mahatma_Gandhi'.
Edge Rewiring: Rewired 45 edges.


...


--- Applying Relation Swapping (RS) ---
Relation Swapping: Swapped relations of 30 edges.

--- Applying Relation Replacement (RR) ---
Relation Replacement: Replaced relations of 30 edges.

--- Applying Edge Rewiring (ER) ---
No available entities to rewire from entity 'Reformation'.
No available entities to rewire from entity 'Reformation'.
No available entities to rewire from entity 'Reformation'.
Failed to rewire edge from entity 'Parlement_de_Normandie' to 'Basque_language'.
Failed to rewire edge from entity 'Basques' to 'Priesthood_of_all_believers'.
Failed to rewire edge from entity 'Beeldenstorm' to 'Primož_Trubar'.
Edge Rewiring: Rewired 30 edges.

--- Applying Edge Deletion (ED) ---
Edge Deletion: Deleted 30 edges.


Perturbing Graphs:   5%|▌         | 42/792 [55:07<18:38:44, 89.50s/it] 


--- Applying Relation Swapping (RS) ---
Relation Swapping: Swapped relations of 48 edges.

--- Applying Relation Replacement (RR) ---
Relation Replacement: Replaced relations of 49 edges.

--- Applying Edge Rewiring (ER) ---
No available entities to rewire from entity 'Roman_Empire'.
No available entities to rewire from entity 'Roman_Empire'.
No available entities to rewire from entity 'Roman_Empire'.
Failed to rewire edge from entity 'Thing' to 'Prefect'.
Failed to rewire edge from entity 'Roman_Senate' to 'Roman_Catholic_Archdiocese_of_Clermont'.
Failed to rewire edge from entity 'Cadiz' to 'Roman_Republic'.
No available entities to rewire from entity 'Roman_Empire'.
No available entities to rewire from entity 'Roman_Empire'.
No available entities to rewire from entity 'Roman_Empire'.
Failed to rewire edge from entity 'Punic_Wars' to 'Thing'.
No available entities to rewire from entity 'Roman_Empire'.
No available entities to rewire from entity 'Roman_Empire'.
Failed to rewire edge 

Perturbing Graphs:   5%|▌         | 43/792 [56:47<19:17:08, 92.70s/it]


--- Applying Relation Swapping (RS) ---
Relation Swapping: Swapped relations of 44 edges.

--- Applying Relation Replacement (RR) ---
